# Kathmandu Youth Fashion Forecaster — EDA & Starter Notebook

This notebook helps you explore the synthetic 1-year datasets (sales, social, Google Trends), clean and filter to the cohort (Kathmandu, male, ages ~17–25), visualize seasonality, extract hashtags, merge trends, and run a quick Prophet demo for one category.

Place this notebook in your project root `MENS_FASHION_FORECAST/notebooks/` so the `data/` folder is at the same level.

In [1]:
# 0. Setup - run once
import os, sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

print("Python executable:", sys.executable)
print("Working dir:", os.getcwd())
print("\nFiles in data/:")
for f in sorted(os.listdir("../data") if os.path.exists("../data") else os.listdir("data")):
    print("-", f)


Python executable: c:\Users\shahi\Desktop\Final Year plan\Mens_Fashion_Forecast\venv\Scripts\python.exe
Working dir: c:\Users\shahi\Desktop\Final Year plan\Mens_Fashion_Forecast\notebooks

Files in data/:
- .gitignore
- google_trends_1yr_nepal.csv
- sales_data_1yr_nepal.csv
- social_media_1yr_nepal.csv
